## Đổi tên file ảnh

In [3]:
import os

folder_path = "./Flower/"
subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]


for subfolder in subfolders:
    path = os.path.join(folder_path, subfolder)
    files = os.listdir(path)
    files.sort()
    new_prefix =  subfolder
    
    for i, file_name in enumerate(files):
        new_name = new_prefix + str(i+1).zfill(2) + ".png"
        os.rename(os.path.join(path, file_name), os.path.join(path, new_name))

## Tách đặc trưng lưu vào file

In [5]:

from Feature import Feature
from PIL import Image
import numpy as np
import json

import os
folder_path = './Flower/'
subfolders = [f.name for f in os.scandir(folder_path) if f.is_dir()]

feature_data = []
feature = Feature()
for subfolder in subfolders:
    # Lấy danh sách các tệp trong thư mục
    files = os.listdir(os.path.join(folder_path, subfolder))
    
    for file_name in files:
        data = {}
        data['label'] = subfolder
        # Đọc ảnh
        img = Image.open(os.path.join(folder_path, subfolder, file_name))
        img = np.array(img)
        # Tính toán đặc trưng
        color_feature = feature.color_histogram(img)
        hog_feature = feature.hog(img)
        data['color_feature'] = color_feature.tolist()
        data['hog_feature'] = hog_feature.tolist()
        feature_data.append(data)

data = {}
data['data'] = feature_data

# Lưu biến feature_data ra file JSON
with open('data.json', 'w') as f:
    json.dump(data, f)

## Test

In [6]:

import json
import numpy as np
from PIL import Image
from Feature import Feature

# Đọc dữ liệu từ file JSON
with open('data.json', 'r') as f:
    data = json.load(f)

# Lấy dữ liệu'
feature_data = data['data']


image1 = Image.open('./Flower/hoa cúc huân chương/hoa cúc huân chương05.png')
image1 = np.array(image1)

feature = Feature()
color_feature = feature.color_histogram(image1)
hog_feature = feature.hog(image1)

list_distance = []

for data in feature_data:
    label = data['label']
    color_feature2 = np.array(data['color_feature'])
    hog_feature2 = np.array(data['hog_feature'])
    
    distance_color = feature.distanceEuclidean(color_feature, color_feature2)
    distance_hog = feature.distanceEuclidean(hog_feature, hog_feature2)
    distance = distance_color + distance_hog
    
    list_distance.append((label, distance))


list_distance.sort(key=lambda x: x[1])
list_distance = list_distance[:5]
print(list_distance)

[('hoa cúc huân chương', 0.0), ('hoa đồng tiền', 14.369073935009865), ('hoa cúc huân chương', 14.8119505883653), ('hoa bướm', 14.85940439067955), ('hoa hướng dương', 14.960741433073606)]
